In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
import os
from scipy.stats import mstats

# Data Loading and Manipulation

In [2]:
data = pd.read_csv("/Users/beneverman/Documents/Coding/QuantHive/IDVF-Oxford-v1/data/processed-5yr-93-minute/65min.csv", index_col=0)
# assuming 6 65-minute periods per day
back_day = 6*20 # 20 days
window_length = 6*250 # 250 days
train_size = 6*1000 # 1000 days

data.ffill(inplace=True)
data.bfill(inplace=True)
assert data.isna().sum().sum() == 0

data['datetime'] = pd.to_datetime(data['datetime'], utc=True)
data.set_index('datetime', inplace=True)

namelist = data.columns.tolist()

def rv(series: pd.Series, window: int) -> pd.Series:
    """
    Realized volatility is defined in [Volatility Forecasting with Machine Learning
    and Intraday Commonality](https://arxiv.org/pdf/2202.08962.pdf) as:

    $$RV_{i,t}(h)=\log(\sum_{s=t-h+1}^{t}r^2_{i,s})$$
    """
    assert window > 0, "Window must be greater than 0"
    fuzz = 1e-16
    log_returns = np.log(series).diff() # log returns
    sum_of_squares = log_returns.rolling(window=window).apply(lambda x: np.sum(x**2), raw=True)
    rv = np.log(sum_of_squares + fuzz)
    assert rv.isna().sum() == window, "RV should have NaNs at the beginning" # ? should have one nan from logret and window - 1 from rolling = window
    return rv

for ind in namelist:
    data[ind + "_logvol"] = rv(data[ind], window_length)

date = data.index

## Preprocessing

In [3]:
class Preprocess:
    # ! They put all the logic in the class constructor - i would prefer to have a separate function for each step
    def __init__(self, input, target, back_day = list(range(0,15)), forward_day = 1):
        # this liss(range(0,15)), forward_day = 1 seem to correspond to the lookback window and the forecast horizon
        # ! input is a list of dataframes, for example [price,volatility] with index as the same as target.
        # list of dfs holds all the results, target is the actual "input" column
        
        # ! Section 1 - make incrementally shifted seqences for each df in the input list
        self.x = []
        for df in input:
            # Shift the dataframe by each value in back_day and concatenate along columns
            # ! detailed explanation below
            shifted_df = pd.concat(
                list(map(lambda n: df.shift(n), back_day)), axis=1
            ).reset_index(drop=True).loc[:, ::-1]
            self.x.append(np.expand_dims(np.array(shifted_df), axis=2)) # Expand dimensions to make it compatible for future concatenation

        self.x = np.concatenate(tuple(self.x), axis=2) # Concatenate all processed input data along the last axis

        # ! X shape = (7516, 15, 1), rows / columns / channels
        # ! X shape = (number of agg bars) / (back day list len) / (#dfs in input list)

        # ! Section 2 - make the target, mask, and date
        idx1 = [~np.any(np.isnan(p)) for p in self.x] # Create an index mask where none of the elements in the x dataframes are NaN
        # ! for each row in x (which includes the row data from all dfs), if any of the values are NaN, then return False, else return True (therefore the length of idx will be the number of rows)
        self.y = target.shift(-forward_day) # Shift the target by forward_day to align with predictor variables
        self.y = pd.DataFrame(self.y).reset_index(drop=True) # Reset index to align with self.x (i removed the double parentheses around self.y)
        self.idx2 = self.y.notna().all(axis=1) # simple mask all rows where there are no NaNs (i.e. all values are present) note that this is notna, not isna like before. So this is the opposite of the previous mask
        self.idx = np.logical_and(idx1, self.idx2) # Combine the two index masks (element-wise and)
        # ! final mask "idx" is of shape (rows, )

        self.x = self.x[self.idx] # Filter x and y data based on combined index mask
        self.y = np.array(self.y[self.idx].reset_index(drop=True)) # Filter date based on combined index mask, make it an array
        
        # ! Section 3 - make the date
        self.idx = data.index[self.idx] #! this is weird naming convention, because now self.idx is the date index from the data df, not a mask anymore

In [4]:
def normalize_V2(x, levels=[0.01, 0.01]):
    """New windsorize function that works with 2D and 3D arrays, updated for readability"""
    # Define a function to winsorize a 1D array
    def winsorize_1d(arr):
        return mstats.winsorize(arr, levels)

    # Determine the axis along which to apply the winsorize function
    # - For 2D arrays, apply along axis 0 (columns)
    # - For 3D arrays, apply along axis 1 (rows within each 2D slice)
    axis = 1 if len(x.shape) == 3 else 0

    # Apply the winsorize function along the specified axis
    y = np.apply_along_axis(winsorize_1d, axis, x)

    return y

In [5]:
class RollingPredict:
    def __init__(self, keywords = ['XVZ_volatility'], back_day = list(range(0, 15)), lr=0.001):
        self.back_day = back_day # list of days to look back
        self.lr = lr # learning rate
        self.keywords = keywords

        first_df = [data[namelist[0]+'_logvol']] # temporary list of dataframes (only one element for now)
        self.a = Preprocess(first_df, data[namelist[0]+'_logvol'], back_day = back_day) # preprocess the data

        for ind in namelist[1:]:
            temp = [data[ind + '_logvol']]
            temp_a = Preprocess(temp, data[ind + '_logvol'], back_day = back_day) 

            # ! these lines essentially concat onto the first_df object stored in self.a
            # ! So theyre updating the x, y, and idx attributes of self.a
            # ! not sure why they did this
            self.a.x = np.concatenate([self.a.x, temp_a.x], axis=0) # concatenate the x data (predictor variables)
            self.a.y = np.concatenate([self.a.y, temp_a.y], axis=0) # concatenate the y data (target variable)
            self.a.idx = np.concatenate([self.a.idx, temp_a.idx], axis=0) # concatenate the date data

    def train(self, train_index, predict_index, lr, names, Epoch_num = 300, pre=True):

        # ! Need to figure out how all this works ========================
        temp_train_start = np.where(self.a.idx == train_index[0]) # get the index of the first training date
        temp_index_train = [] # list of indices for training data

        for i in temp_train_start[0]:
            temp_index_train.extend(list(range(i, i + len(train_index)))) # add the indices for the training data

        temp_predict_start = np.where(self.a.idx == predict_index[0]) # get the index of the first prediction date
        temp_index_predict = []

        for i in temp_predict_start[0]:
            temp_index_predict.extend(list(range(i, i + len(predict_index)))) # add the indices for the prediction data
        
        train_x = self.a.x[temp_index_train] # get the training predictor data
        train_y = self.a.y[temp_index_train] # get the training target data
        test_x = self.a.x[temp_index_predict] # get the test predictor data
        test_y = self.a.y[temp_index_predict] # get the test target data

        # ! ===============================================================
        